In [ ]:
import slacker
s = slacker.Slacker("") # Slack api key goes here

In [ ]:
%pylab inline
# setup, import modules
import mpld3

import json

import nltk
import stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

mpld3.enable_notebook()

In [ ]:
channel_response = slacker.channels.list()

In [ ]:
json.dump(channel_response.body, open("data/channel_data.json", "w"))

In [ ]:
channels = json.load(open("data/channel_data.json"))

In [ ]:
channel_to_message = json.load(open("channel_message.json"))

In [ ]:
channels = channel_response.body["channels"]

In [ ]:
pylab.hist([len(x["members"]) for x in channels], bins=range(0, 1500, 10))

In [ ]:
# Get the number of channels per member, then display it as a histogram
from collections import Counter
import itertools

channels_per_member = Counter(itertools.chain.from_iterable(channel["members"] for channel in channels if not channel.get("is_general", False)))
pylab.hist([x[1] for x in channels_per_member.items() if x[0] not in irregular_users], bins=range(105))

In [ ]:
# Collect all of the channels and all of the messages (in order) in that channel.
from collections import namedtuple
from collections import defaultdict
import time

Message = namedtuple("Message", ["channel", "subtype", "user", "time_sent", "message"])


channel_to_message = defaultdict(list) 
for channel in channels:
    print "started channel", channel["name"]
    has_more = True
    latest = time.time()
    while has_more:
        history_response = s.channels.history(channel["id"], latest=latest, count=1000)
        for message in history_response.body["messages"]:
            channel_to_message[channel["name"]].append(message)
        has_more = history_response.body["has_more"]
        if has_more:
            latest = message["ts"]
    print "finished channel", channel["name"]   

In [ ]:
users_response = s.users.list()


In [ ]:
json.dump(users_response.body, open("data/user_data.json", "w"))

In [ ]:
# Some useful user groupings and data structures

user_ids_to_names = {x["id"]: x["name"] for x in users_response.body["members"]}
user_names_to_ids = {x["name"]: x["id"] for x in users_response.body["members"]}
restricted_users = {x["id"] for x in users_response.body["members"] if x.get("is_restricted", False)}
ultra_restricted_users = {x["id"] for x in users_response.body["members"] if x.get("is_ultra_restricted", False)}
bots = {x["id"] for x in users_response.body["members"] if x.get("is_bot", False)}
irregular_users = restricted_users | ultra_restricted_users | bots

In [ ]:
print "total regular (non-restricted, non-ultra-restricted, non-bot) users:", len([x for x in users_response.body["members"] if x["id"] not in irregular_users])

In [ ]:
channels_per_regular_user = Counter({x[0]: x[1] for x in channels_per_member.items() if x[0] not in irregular_users})
most_channels = [(user_ids_to_names[x[0]], x[1]) for x in channels_per_regular_user.most_common(15)]

average_num_channels = sum(channels_per_regular_user.values())/len(channels_per_regular_user)
median_num_channels = sorted(channels_per_regular_user.values())[len(channels_per_regular_user)/2]
mode_num_channels = Counter(channels_per_regular_user.values()).most_common(1)[0]
print "users with the most channels: ", most_channels
print "average number of channels:", average_num_channels
print "median number of channels:", median_num_channels
print "mode number of channels: {} (with {} users)".format(mode_num_channels[0], mode_num_channels[1])

In [ ]:
print "largest channels: ", sorted([(len(values), key) for (key, values) in channel_to_message.items()], reverse=True)[:5]

In [ ]:
# The next few cells are an experiment to try to get topics per channel. Not very effective yet.

import nltk
import stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

chan_to_text = {}
texts = []

for i, (channel, messages) in enumerate(channel_to_message.iteritems()):
    text = []
    for message in messages:
        if "subtype" not in message and not message["text"].startswith("/"):
            text.extend(tokenize(message["text"]))
    chan_to_text[channel] = text
    if i and not i % 100:
        print "finished {} of {}".format(i, len(channel_to_message))

In [ ]:

dictionary = corpora.Dictionary(chan_to_text.values())

print "made dictionary"
chan_to_bow = {channel: dictionary.doc2bow(texts) for channel, texts in chan_to_text.iteritems()}
print "made chan_to_bow"
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(chan_to_bow.values(), num_topics=300, id2word = dictionary, passes=20)

In [ ]:
en_words = set(stop_words.get_stop_words('en'))

p_stemmer = PorterStemmer()
import re
punct_pattern = r""
link_pattern = r"<[^\s]+>:?"
emoji_pattern = r":[^\s]+:"
whitespace_condenser = re.compile(r"\s+")
matcher = re.compile("|".join([link_pattern, emoji_pattern]))
def tokenize(message):
    subbed = matcher.sub("", message.lower())
    condensed = whitespace_condenser.sub(" ", subbed)
    stopped = (x for x in condensed.split() if x and x not in en_words)
    return list(stopped)#[p_stemmer.stem(x) for x in stopped]

In [ ]:
[(dictionary[topic], value) for topic, value in ldamodel[chan_to_bow["general"]]]

In [ ]:
texts = []
for message in channel_to_message["general"]:
    if "subtype" not in message and not message["text"].startswith("/"):
        texts.append(tokenize(message["text"]))
dictionary_swift = corpora.Dictionary(texts)
ldamodel_swift = gensim.models.ldamodel.LdaModel([dictionary_swift.doc2bow(text) for text in texts], num_topics=10, id2word=dictionary_swift, passes=20)
ldamodel_swift.show_topics()

In [ ]:
gensim.models.ldamodel.LdaModel([dictionary_kiwi.doc2bow(text) for text in texts], num_topics=10, id2word=dictionary_kiwi, passes=20).show_topics

In [ ]:
for message in channel_to_message["general"][:15]:
    if "subtype" not in message and not message['text'].startswith("/"):
        message = message["text"]
        subbed = matcher.sub("", message.lower())
        condensed = whitespace_condenser.sub(" ", subbed)
        stopped = [x for x in condensed.split() if x and x not in en_words]
        stemmed = [p_stemmer.stem(x) for x in stopped]
        print message
        print subbed
        print condensed
        print stopped
        print stemmed
        print "---------------"